In [1]:
import pandas as pd
import cv2
import face_recognition
import numpy as np
from collections import Counter

# 질문 db 불러오기
who_df = pd.read_csv("./dataset/query_who.csv", encoding='cp949')
place_df = pd.read_csv("./dataset/query_place.csv") 
when_df = pd.read_csv("./dataset/query_when.csv") 
what_df = pd.read_csv("./dataset/query_what.csv") 
where_df = pd.read_csv("./dataset/query_where.csv") 
entity_df = pd.read_csv("./dataset/entity_dataset.csv", encoding='cp949')

In [2]:
# entity 항목 불러오기
entity_who = entity_df.loc[0, 'entity'].split(', ')
entity_place = entity_df.loc[1, 'entity'].split(', ')
entity_time = entity_df.loc[2, 'entity'].split(', ')
entity_object = entity_df.loc[3, 'entity'].split(', ')
entity_body = entity_df.loc[4, 'entity'].split(', ')

In [3]:
# 언제에 해당하는 entity에 시간 추가
for i in range(1, 13):
    tmp = str(i) + '시'
    entity_time.append(tmp)

## 본문

#### 재입력 기능

In [4]:
# 장애인이 틀렸다고 한 부분만 다시 입력하는 기능
# 입력한 내용인 db에 있는지 확인
# ex) 장애인이 때린 사람을 아저씨, 활동보조인으로 입력
# db에 '아저씨', '활동보조인'이 없을 경우 재입력 요구
def __anschk__(anses, len, df):

    cnt = 0
    
    for a in anses:
        if a in df:
            cnt += 1
    
    if cnt == len:
        return True

    return False

### 전체 함수

#### 누가

In [5]:
expected_A1 = {'네', '아니오', '잘모르겠어요'}
# 전체다 모르겠어요로 고칠 것

# 누가
def __who__():
    
    no_ans = '모르는 사람' 
    temp = who_df.query('label == [0]').sample(n=5) # csv에서 랜덤으로 라벨이 0인 질문 5개 추출
    temp = temp.loc[:, 'Q'].tolist() # 뽑은 질문을 리스트로 변환
    pass_cnt = 0 # 아니오를 2번 이상 답할 경우 대상을 모르는 것으로 알고 다음 질문을 넘어감
                 # 예를 들어 누가 때렸는지 알아요? 라는 질문에 2번 아니오라 답하면 때린 인물이 기억이 안나는 것으로 보고 '어디서' 질문으로 넘어감
    step_pass = False
                 # 다음 label 질문으로 넘어가도 되는지 확인하는 bool

    for q in temp:
        if step_pass:
            break
        print(q)
        q_pass = False
        # label은 그대로하고 질문만 새롭게 출력하는 것을 확인하는 bool
        while not q_pass:
            ans = input()
            print(ans)
            if ans == '네':
                q_pass = True
                step_pass = True
            elif ans == '아니오':
                pass_cnt += 1
                if pass_cnt == 2:
                    return no_ans
                elif q == temp[-1]:
                    print('기억이 나지 않나 보네요.\n괜찮습니다. 나중에라도 기억이 나면 알려주세요.')   
                    return no_ans                   
                print('장애인 분을 위해서 때린 사람이 누구인지 확실하게 알 필요가 있어요.\
                    \n질문이 이해가 안 가시면 잘 모르겠다고 답해주세요.\
                    \n기억이 나지 않는다면 한 번 더 아니오를 눌러주세요.\
                    \n아니오를 한 번 더 누르면 다음 질문을 할게요.')
                print('질문을 다르게 해볼게요.')
                q_pass = True
            elif ans == '잘모르겠어요':
                print('질문을 다르게 해볼게요.')
                q_pass = True

            # aac로 답하기 때문에 답변에 오타가 날 경우를 제외했다.
            '''    
            elif ans not in expected_A1:
                print('답변은 네, 아니오, 잘모르겠어요 중 3가지로 해주세요')   
            '''
        if q == temp[-1] and step_pass == False:
            print('기억이 나지 않나 보네요.\n괜찮습니다. 나중에라도 기억이 나면 알려주세요.')
            return no_ans

    temp = who_df.query('label == [1]').sample(n=5) # 라벨이 1인 질문 중 5개를 랜덤으로 추출
    temp = temp.loc[:, 'Q'].tolist()
    step_pass = False

    for q in temp:
        if step_pass:
            break
        print(q)
        q_pass = False
        while not q_pass:
            ans = input()
            print(ans)

            if ans == '잘모르겠어요':
                print('질문을 다르게 해볼게요.')
                q_pass = True
                
            else:
                ans_list = ans.split(' ')
                ans_pass = __anschk__(ans_list, len(ans_list), entity_who)
                if ans_pass:
                    whotmp = ans_list[0]
                    for i in ans_list[1:]:
                        whotmp = whotmp +', '+ i
                    print('때린 사람이 '+whotmp+'였군요.\n이제부터 '+whotmp+'가 어디서 때렸는지 물어볼게요')
                    q_pass = True
                    step_pass = True

    return whotmp

#### 어디서

In [6]:
# 어디서
def __place__(who):

    no_ans = '모르는 장소'
    temp = place_df.query('label == [0]').sample(n=5)
    temp = temp.loc[:, 'Q'].tolist()
    step_pass = False
    pass_cnt = 0
    
    for q in temp:
        if step_pass:
            break
        q = q.replace('{tag_people}', who)
        print(q)
        q_pass = False
        while not q_pass:
            ans = input()
            print(ans)
            if ans == '네':
                q_pass = True
                step_pass = True
            elif ans == '아니오':
                pass_cnt += 1
                if pass_cnt == 2:
                    return no_ans
                elif q == temp[-1]:
                    print('기억이 나지 않나 보네요.\n괜찮습니다. 나중에라도 기억이 나면 알려주세요.')   
                    return no_ans                    
                print('장애인 분을 위해서 '+who+'가 어디서 때렸는지 확실하게 알 필요가 있어요.\
                    \n질문이 이해가 안 가시면 잘 모르겠다고 답해주세요.\
                    \n기억이 나지 않는다면 한 번 더 아니오를 눌러주세요.\
                    \n아니오를 한 번 더 누르면 다음 질문을 할게요.')
                print('질문을 다르게 해볼게요.')
                q_pass = True
            elif ans == '잘모르겠어요':
                print('질문을 다르게 해볼게요.')
                q_pass = True

        if q == temp[-1] and step_pass == False:
            print('기억이 나지 않나 보네요.\n괜찮습니다. 나중에라도 기억이 나면 알려주세요.')
            return no_ans

    temp = place_df.query('label == [1]').sample(n=5)
    temp = temp.loc[:, 'Q'].tolist()
    step_pass = False

    for q in temp:
        if step_pass:
            break
        q_pass = False
        q = q.replace('{tag_people}', who)
        print(q)
        step_pass = False
        while not q_pass:
            ans = input()
            print(ans)

            if ans == '잘모르겠어요':
                print('질문을 다르게 해볼게요.')
                q_pass = True
                
            else:
                ans_list = ans.split(' ')
                ans_pass = __anschk__(ans_list, len(ans_list), entity_place)
                if ans_pass:
                    whentmp = ans_list[0]
                    for i in ans_list[1:]:
                        whentmp = whentmp +', '+ i
                print(who+'가 때린 장소가 '+whentmp+'였군요.\n이제부터'+who+'가 언제 때렸는지 물어볼게요')
                q_pass = True
                step_pass = True
    
    return whentmp

#### 언제

In [7]:
def __when__(who, place):

    no_ans = '모르는 시간'
    temp = when_df.query('label == [0]').sample(n=5)
    temp = temp.loc[:, 'Q'].tolist()
    step_pass = False
    pass_cnt = 0

    for q in temp:
        if step_pass:
            break
        q = q.replace('{tag_people}', who).replace('{tag_place}', place)
        print(q)
        q_pass = False
        while not q_pass:
            ans = input()
            print(ans)
            if ans == '네':
                q_pass = True
                step_pass = True
            elif ans == '아니오':
                pass_cnt += 1
                if pass_cnt == 2:
                    return no_ans
                elif q == temp[-1]:
                    print('기억이 나지 않나 보네요.\n괜찮습니다. 나중에라도 기억이 나면 알려주세요.')   
                    return no_ans                   
                print('장애인 분을 위해서'+who+'가 언제 때렸는지 확실하게 알 필요가 있어요.\
                    \n질문이 이해가 안 가시면 잘 모르겠다고 답해주세요.\
                    \n기억이 나지 않는다면 한 번 더 아니오를 눌러주세요.\
                    \n아니오를 한 번 더 누르면 다음 질문을 할게요.')
                print('질문을 다르게 해볼게요.')
                q_pass = True
            elif ans == '잘모르겠어요':
                print('질문을 다르게 해볼게요.')
                q_pass = True
                
        if q == temp[-1] and step_pass == False:
            print('기억이 나지 않나 보네요.\n괜찮습니다. 나중에라도 기억이 나면 알려주세요.')
            return no_ans

    temp = when_df.query('label == [1]').sample(n=5)
    temp = temp.loc[:, 'Q'].tolist()
    step_pass = False

    for q in temp:
        if step_pass:
            break
        q_pass = False
        q = q.replace('{tag_people}', who).replace('{tag_place}', place)
        print(q)
        step_pass = False
        while not q_pass:
            ans = input()
            print(ans)

            if ans == '잘모르겠어요':
                print('질문을 다르게 해볼게요.')
                q_pass = True
                
            else:
                ans_list = ans.split(' ')
                ans_pass = __anschk__(ans_list, len(ans_list), entity_time)
                if ans_pass:
                    whentmp = ans_list[0]
                    for i in ans_list[1:]:
                        whentmp = whentmp +', '+ i
                print(who,'가 ',place,'에서 장애인님을 때렸군요. 때린 시간은 ',whentmp,'였구요\
                    \n이제부터',who,'가 무엇으로 때렸는지 물어볼게요')
                q_pass = True
                step_pass = True

    return whentmp

#### 무엇으로

In [8]:
def __what__(who, place, when):

    no_ans = '모르는 물건'
    temp = what_df.query('label == [0]').sample(n=5)
    temp = temp.loc[:, 'Q'].tolist()
    step_pass = False
    pass_cnt = 0

    for q in temp:
        if step_pass:
            break
        q = q.replace('{tag_people}', who).replace('{tag_place}', place).replace('{tag_time}', when)
        print(q)
        q_pass = False
        while not q_pass:
            ans = input()
            print(ans)
            if ans == '네':
                q_pass = True
                step_pass = True
            elif ans == '아니오':
                print('장애인님을 때릴 때 도구를 사용하지 않았군요.')
                return '맨손'
                
            elif ans == '잘모르겠어요':
                print('질문을 다르게 해볼게요.')
                q_pass = True

        if q == temp[-1] and step_pass == False:
            print('기억이 나지 않나 보네요.\n괜찮습니다. 나중에라도 기억이 나면 알려주세요.')
            return no_ans

    temp = what_df.query('label == [1]').sample(n=5)
    temp = temp.loc[:, 'Q'].tolist()
    step_pass = False

    for q in temp:
        if step_pass:
            break
        q_pass = False
        q = q.replace('{tag_people}', who).replace('{tag_place}', place).replace('{tag_time}', when)
        print(q)
        step_pass = False
        while not q_pass:
            ans = input()
            print(ans)
            if ans == '잘모르겠어요':
                print('질문을 다르게 해볼게요.')
                q_pass = True
                
            else:
                ans_list = ans.split(' ')
                ans_pass = __anschk__(ans_list, len(ans_list), entity_object)
                if ans_pass:
                    whattmp = ans_list[0]
                    for i in ans_list[1:]:
                        whattmp = whattmp +', '+ i
                print(who+'가 '+place+'에서 '+whattmp+'로 장애인님을 때렸군요. 때린 시간은 '+when+'이였구요\
                    \n이제부터'+who+'가 '+whattmp+'로 어디를 때렸는지 물어볼게요')
                q_pass = True
                step_pass = True
            
    return whattmp

#### 어디를

In [9]:
def __where__(who, place, when, what):

    no_ans = '모르는 부위'
    temp = where_df.query('label == [0]').sample(n=5)
    temp = temp.loc[:, 'Q'].tolist()
    step_pass = False
    pass_cnt = 0
    
    for q in temp:
        if step_pass:
            break
        q = q.replace('{tag_people}', who).replace('{tag_place}', place).replace('{tag_time}', when).replace('{tag_object}', what)
        print(q)
        q_pass = False
        while not q_pass:
            ans = input()
            print(ans)
            if ans == '네':
                q_pass = True
                step_pass = True
                break
            elif ans == '아니오':
                pass_cnt += 1
                if pass_cnt == 2:
                    return no_ans
                elif q == temp[-1]:
                    print('기억이 나지 않나 보네요.\n질문은 이걸로 끝입니다.')   
                    return no_ans 
                print('저희가 장애인 분을 확실히 도와드리고 싶어요.\
                    \n도와드리기 위해서는 ',who,'가 어디를 때렸는지 알아야해요.\
                    \n기억이 나지 않거나 질문이 이해가 안 가시면 잘 모르겠다고 답해주세요.')
                print('질문을 다시 할게요.')
                q_pass = True
            elif ans == '잘모르겠어요':
                print('질문을 다르게 해볼게요.')
                q_pass = True

        if q == temp[-1] and step_pass == False:
            print('기억이 나지 않나 보네요.\n질문은 이걸로 끝입니다.')
            return no_ans

    temp = where_df.query('label == [1]').sample(n=5)
    temp = temp.loc[:, 'Q'].tolist()
    step_pass = False

    for q in temp:
        if step_pass:
            break
        q_pass = False
        q = q.replace('{tag_people}', who).replace('{tag_place}', place).replace('{tag_time}', when).replace('{tag_object}', what)
        print(q)
        step_pass = False
        while not q_pass:
            ans = input()
            print(ans)
            if ans == '잘모르겠어요':
                print('질문을 다르게 해볼게요.')
                q_pass = True    
            else:
                ans_list = ans.split(' ')
                ans_pass = __anschk__(ans_list, len(ans_list), entity_body)
                if ans_pass:
                    wheretmp = ans_list[0]
                    for i in ans_list[1:]:
                        wheretmp = wheretmp +', '+ i
                print(who+'가 '+what+'(으)로 '+wheretmp+'를 때렸군요.\
                    \n질문은 이걸로 끝입니다.\n대답해주신걸 다시 확인해볼게요.')
                step_pass = True
                q_pass = True

    return wheretmp

#### 재입력 기능

In [10]:
def __check__(who, place, when, what, where):
    chksen = who+'가 '+place+'에서 '+when+'에 '+what+'으로 장애인님의 '+where+'를 때린게 맞나요?'
    print(chksen)
    step_pass = False
    while not step_pass:
        ans = input()
        print(ans)
        if ans == '네':
            print('신고가 정상적으로 접수되었습니다. 금방 연락을 드릴게요.')
            step_pass = True
            return who, place, when, what, where, True
        elif ans == '아니오':
            print('어디가 틀린건지 얘기해주세요.')
            wrong = input()
            fix_list = wrong.split(' ')
            for i in fix_list:
                if i == '누가':
                    who = __who__()
                if i == '어디서':
                    place = __place__(who)
                if i == '언제':
                    when = __when__(who, place)
                if i == '무엇으로':
                    what = __what__(who, place, when)
                if i == '어디를':
                    where = __where__(who, place, when, what)
            return who, place, when, what, where, False

## 실행

#### 얼굴 인식

In [11]:
video_capture = cv2.VideoCapture(0)

# 얼굴 학습
Ko_image = face_recognition.load_image_file("./test_c/KakaoTalk_20221127_184457705.jpg")
Ko_face_encoding = face_recognition.face_encodings(Ko_image)[0]

Kwon_image = face_recognition.load_image_file("./test_c/KakaoTalk_20221129_205044499.jpg")
Kwon_face_encoding = face_recognition.face_encodings(Kwon_image)[0]

Jang_image = face_recognition.load_image_file("./test_c/KakaoTalk_20221129_204921270.jpg")
Jang_face_encoding = face_recognition.face_encodings(Jang_image)[0]

Baek_image = face_recognition.load_image_file("./test_c/KakaoTalk_20221129_204941840.jpg")
Baek_face_encoding = face_recognition.face_encodings(Baek_image)[0]

known_face_encodings = [
    Ko_face_encoding,
    Kwon_face_encoding,
    Jang_face_encoding,
    Baek_face_encoding
]
known_face_names = [
    "ID: 2015111766",
    "ID: 2019113394",
    "Jang Jiwook",
    "Baek Jongrok"
]

face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
cnt = 0
chk_list = []

while True:

    ret, frame = video_capture.read()

    if process_this_frame:
        # 영상 설정
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        rgb_small_frame = small_frame[:, :, ::-1]
    
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    for (top, right, bottom, left), name in zip(face_locations, face_names):
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # 인식된 얼굴 주변을 박스로 표시
        cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)

        # 얼굴에 표시
        # BGR 순으로 표시된다.
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (255, 191, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)
        chk_list.append(name)

    # 결과창 표시
    cv2.imshow('Video', frame)
    
    # 일정 빈도 이상 인식되면 얼굴 확인 완료
    if chk_list:
        counter = Counter(chk_list)
        if counter.most_common()[0][1] == 15:
            T_name = name
            break
    

    # q를 눌러 종료
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    if key == ord('p'):
        cv2.waitKey(-1) #wait until any key is pressed
    

# 사용하고 있던 웹캠 놓기
video_capture.release()
cv2.destroyAllWindows()

#### 질문 시작

In [12]:
print('안녕하세요. '+T_name+'님')
print('질문을 시작하겠습니다.\n최대한 모두 대답해주세요')

print('-----------------------------------누가---------------------------------------')
tag_who = __who__()
print('-----------------------------------어디서---------------------------------------')
tag_place = __place__(tag_who)
print('-----------------------------------언제---------------------------------------')
tag_when = __when__(tag_who, tag_place)
print('-----------------------------------무엇으로---------------------------------------')
tag_what = __what__(tag_who, tag_place, tag_when)
print('-----------------------------------어디를---------------------------------------')
tag_where = __where__(tag_who, tag_place, tag_when, tag_what)
print('-----------------------------------질문 종료 및 재확인---------------------------------------')

ok = False
while not ok:
    tag_who, tag_place, tag_when, tag_what, tag_where, ok = __check__(tag_who, tag_place, tag_when, tag_what, tag_where)

안녕하세요. ID: 2015111766님
질문을 시작하겠습니다.
최대한 모두 대답해주세요
-----------------------------------누가---------------------------------------
때린 사람이 누구인지 알아야해요. 때린 사람 기억나요?

